# Create Slice for Fabric Rack
This is the slice creation for the ```OSPF``` tutorial. This is the first step of the assignment, to create the slice on fabric using fablib.
 

## 1. Set up the Experiment

In this section you will use the Fablib manager to create a new slice that is composed of 4 nodes for the OSPF Assignment.
### 1.1 Reserve Resources
In this section we will declare the structure of the slice we will be using.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "OSPF"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

In [ ]:
# Setup Network
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    #Node A
    A = slice.add_node(name="ND_A", site=site)
    A.set_image('default_ubuntu_20')
    AP1 = A.add_component(model='NIC_Basic', name="APort1").get_interfaces()[0] 
    AP2 = A.add_component(model='NIC_Basic', name="APort2").get_interfaces()[0]
    
    #Node B
    B = slice.add_node(name="ND_B", site=site)
    B.set_image('default_ubuntu_20')
    BP1 = B.add_component(model=nicmodel, name="BPort1").get_interfaces()[0] 
    BP2 = B.add_component(model=nicmodel, name="BPort2").get_interfaces()[0]
    
    #Node C
    C = slice.add_node(name="ND_C", site=site)
    C.set_image('default_ubuntu_20')
    CP1 = C.add_component(model=nicmodel, name="CPort1").get_interfaces()[0] 
    CP2 = C.add_component(model=nicmodel, name="CPort2").get_interfaces()[0]
    
    #Node D
    D = slice.add_node(name="ND_D", site=site)
    D.set_image('default_ubuntu_20')
    DP1 = D.add_component(model=nicmodel, name="DPort1").get_interfaces()[0] 
    DP2 = D.add_component(model=nicmodel, name="DPort2").get_interfaces()[0]
    
    Lan1 = slice.add_l2network(name="Lan1", interfaces=[AP1, BP1])
    Lan2 = slice.add_l2network(name="Lan2", interfaces=[BP2, CP1])
    Lan3 = slice.add_l2network(name="Lan3", interfaces=[CP2, DP1])
    Lan4 = slice.add_l2network(name="Lan4", interfaces=[DP2, AP2])
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

### 1.2 Set up the Experiment Network
In this section of the slice creation, once the nodes are active, will assign the network spaces some predefined values that we will use for the assignment

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    A = slice.get_node(name="ND_A") 
    B = slice.get_node(name="ND_B")
    C = slice.get_node(name="ND_C")
    D = slice.get_node(name="ND_D")
    
    subnet1 = IPv4Network("10.1.1.0/24")
    subnet2 = IPv4Network("10.1.2.0/24")
    subnet3 = IPv4Network("10.1.3.0/24")
    subnet4 = IPv4Network("10.1.4.0/24")
    
    AP1 = A.get_interface(network_name="Lan1") 
    AP1.ip_addr_add(addr="10.1.1.1", subnet=subnet1)
    
    BP1 = B.get_interface(network_name="Lan1") 
    BP1.ip_addr_add(addr="10.1.1.2", subnet=subnet1)
    
    BP2 = B.get_interface(network_name="Lan2") 
    BP2.ip_addr_add(addr="10.1.2.1", subnet=subnet2)
            
    CP1 = C.get_interface(network_name="Lan2") 
    CP1.ip_addr_add(addr="10.1.2.2", subnet=subnet2)
    
    CP2 = C.get_interface(network_name="Lan3") 
    CP2.ip_addr_add(addr="10.1.3.1", subnet=subnet3)
    
    DP1 = D.get_interface(network_name="Lan3") 
    DP1.ip_addr_add(addr="10.1.3.2", subnet=subnet3)
    
    DP2 = D.get_interface(network_name="Lan4") 
    DP2.ip_addr_add(addr="10.1.4.1", subnet=subnet4)
            
    AP2 = A.get_interface(network_name="Lan4") 
    AP2.ip_addr_add(addr="10.1.4.2", subnet=subnet4)
    
    A_Device_Name = AP1.get_device_name()
    B_Device_Name = AP2.get_device_name()
    
    for node in slice.get_nodes():
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")
        
except Exception as e:
    print(f"Exception: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
This section of the slice creation, we will add any aditional software, tools or scripts that we need for our experiments.

In [ ]:
# Install Software
INTERFACE_NAME_HOLDER1 = "INTERFACE_NAME_1"
INTERFACE_NAME_HOLDER2 = "INTERFACE_NAME_2"

try:
    for node in slice.get_nodes():
        node.execute(" wget https://github.com/fabric-testbed/teaching-materials/raw/main/assignments/OSPF/scripts/ospf-script.sh")
        #node.upload_file("scripts/ospf-script.sh","ospf-script.sh")
        node.execute(f"sed -i 's/{INTERFACE_NAME_HOLDER1}/{A_Device_Name}/' ospf-script.sh")
        node.execute(f"sed -i 's/{INTERFACE_NAME_HOLDER2}/{B_Device_Name}/' ospf-script.sh")
        node.execute("chmod +x ospf-script.sh")
        node.execute("./ospf-script.sh")
        node.execute("sudo cp /usr/share/doc/quagga-core/examples/vtysh.conf.sample /etc/quagga/vtysh.conf")
        node.execute("sudo cp /usr/share/doc/quagga-core/examples/bgpd.conf.sample /etc/quagga/bgpd.conf")
        node.execute("sudo chown quagga:quagga /etc/quagga/*.conf")
        node.execute("sudo chown quagga:quaggavty /etc/quagga/vtysh.conf")
        node.execute("sudo chmod 640 /etc/quagga/*.conf")
        node.execute("sudo service zebra start")
        node.execute("sudo service ospfd start")
except Exception as e:
    print(f"Fail: {e}")

    

## Continue to The experiment notebook

Once you have completed this notebook you should be able to continue to the OSPF Notebook. You can either open it on the explorer or click [Here](./OSPF.ipynb) to open the next notebook.